In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.optimizers import *
from keras.models import load_model
import matplotlib.pyplot as plt
from music21 import chord, instrument, converter, note, stream
import glob
from tensorflow.keras.utils import to_categorical
import time
import random

print("DONE")

In [ ]:


notes = []
start_time = time.time()
for file in glob.glob("dane_treningowe/midi_clear/*.mid"):
    midi = converter.parse(file)
    parts = instrument.partitionByInstrument(midi)

    if parts:
        for part in parts:
            if part.getInstrument() is not None:
                instrument_name = part.getInstrument().instrumentName
                if instrument_name == "Acoustic Guitar":
                    notes_to_parse = part.recurse()
                    chord_notes = []
                    for element in notes_to_parse:
                        if isinstance(element, note.Note):
                            notes.append(str(element.pitch))
                        elif isinstance(element, chord.Chord):
                            chord_notes = [str(n) for n in element.normalOrder]
                            chord_str = '.'.join(chord_notes)
                            notes.append(chord_str)

with open("output2.txt", "w") as file:
    for note_item in notes:
        file.write(note_item + " ")
end_time = time.time()
elapsed_time = end_time - start_time
elapsed_time_minutes = elapsed_time / 60
print(f"Czas wykonywania programu: {elapsed_time_minutes} minut") 
print("DONE")


In [ ]:
pitchnames = sorted(set(item for item in notes)) # zbiór unikalnych akordów i nut
n_vocab = len(pitchnames) #ilosc unikalnych nut i akordów
#słownik, przypisanie nutom ich odpowiednich liczb
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
print("DONE")

In [ ]:
#wejscia i wyjścia sieci
network_input = []
network_output = []
network = []
seq = 25 # dlugosc sekwencji wejsciowej 10,25,50,100
sequence_length = seq + 1

for i in range(0, len(notes) - sequence_length):  
    sequence = notes[i:i + sequence_length]
    network.append(sequence)

random.shuffle(network)

for j in range(len(network)):
    sequence_in = network[j][:(sequence_length - 1)]
    sequence_out = network[j][sequence_length - 1]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
    n_patterns = len(network_input)


print("DONE")

In [ ]:
# normalizacja danych wejściowych
network_input = np.reshape(network_input, (n_patterns, seq, 1)) 
network_input = network_input / float(n_vocab) # od 0 do 1
network_output = to_categorical(network_output) # one-hot encoding
print("Kształt danych wejściowych (network_input):", network_input.shape)
print("Kształt danych wyjściowych (network_output):", network_output.shape)
# print(network_output)
network_input_train = network_input[: int(0.7*len(network_input))]
network_output_train = network_output[:int(0.7*len(network_output))]

network_input_val = network_input[int(0.7*len(network_input)):int(0.85*len(network_input))]
network_output_val = network_output[int(0.7*len(network_output)):int(0.85*len(network_output))]

network_input_test = network_input[int(0.85*len(network_input)):]
network_output_test = network_output[int(0.85*len(network_output)):]
print("Kształt danych wejściowych treningowych :", network_input_train.shape)
print("Kształt danych wyjściowych treningowych :", network_output_train.shape)
print("Kształt danych wejściowych walidacyjnych :", network_input_val.shape)
print("Kształt danych wyjściowych walidacyjnych :", network_output_val.shape)
print("Kształt danych wejściowych testowych :", network_input_test.shape)
print("Kształt danych wyjściowych testowych :", network_output_test.shape)

In [ ]:
# tworzenie modelu
activation = 'relu'
optimizer = 'rmsprop' # 'rmsprop'Adam
loss='categorical_crossentropy'
num_of_neurons = 20 # 20, 50, 100, 200
num_of_layers = 2
model = Sequential()
model.add(LSTM(num_of_neurons, activation=activation, input_shape=(network_input_train.shape[1], network_input_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(num_of_neurons, activation=activation, return_sequences=False)) 
model.add(Dropout(0.2))
model.add(Dense(n_vocab, activation='softmax')) 
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
model.summary()
print("DONE")

In [ ]:
# checkpoint - zapisanie najlepszego modelu
epochs = 120
batch_size = 100
file_name = f'MODEL_3lstm_{num_of_neurons}neurons_{epochs}epochs_{batch_size}batchsize_{seq}seq_len_{num_of_layers}_num_of_layers.hdf5'
checkpoint = ModelCheckpoint(file_name, save_best_only=True, monitor='accuracy', verbose=0)
print("DONE")

In [ ]:
# trening
history = model.fit(network_input_train, network_output_train, epochs=epochs, batch_size=batch_size, validation_data=(network_input_val, network_output_val), callbacks=[checkpoint])
print("DONE")

In [ ]:
#loaded_model = load_model('MODEL_2lstm_25neurons_25epochs_100batchsize_seq100.hdf5')

#try:
#    history = loaded_model.history
#except AttributeError:
#    print("Model nie posiada historii trenowania.")
#    history = None

if history is not None:
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')

    # Wykres val_accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.title('Model Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Validation Accuracy')
    plt.legend(loc='lower right')

    plt.show()
else:
    print("Brak dostępnej historii trenowania.")


In [ ]:
# generowanie nowych nut
number_of_notes = 200
start = np.random.randint(0, len(network_input) - sequence_length - 1)
#start = 11
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
# model = load_model("MODEL_2lstm_25neurons_400epochs_100batchsize.hdf5")
for note_index in range(number_of_notes):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1)) # normalizacja 
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    pattern = np.append(pattern, index)
    pattern = pattern[1:]
    result = int_to_note[index]
    prediction_output.append(result)    
print("DONE")
print("______________________")
print(prediction_output)
print(len(prediction_output))
print("______________________")


In [ ]:
print(prediction_output)

In [ ]:
# tworzenie nut i akordów
offset = 0  # przesuniecie nut lub akordów (czas)
output_notes = []

for item in prediction_output:
    # szukanie akordów
    if ('.' in item):
        notes_in_chord = item.split('.')
        notes12 = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.AcousticGuitar()
            notes12.append(new_note)
        new_chord = chord.Chord(notes12)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # nuta
    else:
        new_note = note.Note(item)
        new_note.offset = offset
        new_note.storedInstrument = instrument.AcousticGuitar()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5

print("DONE")
print(output_notes)


In [ ]:
# tworzenie pliku midi

file_out = f'MODEL_3lstm_{num_of_neurons}neurons_{epochs}epochs_{batch_size}batchsize_{seq}seq_len_{num_of_layers}_num_of_layers.mid'

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp=file_out)
print("DONE")

In [ ]:
mf = converter.parse(file_out)
mf.show('midi')

In [ ]:
print(prediction_output)